<a href="https://colab.research.google.com/github/abhijit-z/IDS_506_Omicron_Tweets_Analysis/blob/main/Generating_Individual_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Retaining only those tweets that are most likely tweeted by individuals

In [ ]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import os
import re
import string
import nltk

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file1 = pd.read_csv("/content/drive/MyDrive/Omicron Tweet Analytics project/Filter2_res/Further_filtering/df_ff_11212021.csv")
print(file1.columns)

file1.head()

Index(['Unnamed: 0', 'coordinates', 'created_at', 'hashtags', 'place',
       'source', 'user_id', 'user_description', 'user_name', 'id',
       'user_screen_name', 'user_urls', 'user_verified', 'text', 'ind_org',
       'name_plus_desp', 'name_plus_desp_clean', 'new_col2'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,coordinates,created_at,hashtags,place,source,user_id,user_description,user_name,id,user_screen_name,user_urls,user_verified,text,ind_org,name_plus_desp,name_plus_desp_clean,new_col2
0,0,NaN,Sun Nov 21 05:01:57 +0000 2021,NaN,NaN,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",3.064873e+09,"Breaking news, sport, TV, radio and a whole lo...",BalkansTimes.eu,1462285128420544514,BalkansTimesEu,https://www.balkanstimes.eu/,False,Ottawa under fire for changes to COVID-19 bord...,ind,"BalkansTimes.eu Breaking news, sport, TV, radi...",balkanstimeseu breaking news sport tv radio wh...,org
1,1,NaN,Sun Nov 21 05:04:25 +0000 2021,chatwithmark,NaN,"<a href=""http://twitter.com/download/android"" ...",1.272774e+18,#Got7:JB|M|J|JY|YJ|Y|BB|YG| 𓅩 |\nseven or neve...,jelly,1462285749101875202,getdab7,NaN,False,@marktuan #chatwithmark \n\nget i get a hug fr...,ind,jelly #Got7:JB|M|J|JY|YJ|Y|BB|YG| 𓅩 |\nseven o...,jelly got7jbmjjyyjybbyg seven never everever g...,ind
2,2,NaN,Sun Nov 21 05:03:38 +0000 2021,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",1.448159e+18,NaN,SilverSamurai714,1462285552988786695,SilverSamurai85,NaN,False,@Faceplants00 @RWMaloneMD Why? The testing is ...,ind,NaN,NaN,ind
3,3,NaN,Sun Nov 21 05:02:24 +0000 2021,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",1.345052e+18,DvidThomsonB,DavidBxxxx,1462285241763069957,ThomsonBreage,NaN,False,She is Miss BJP 2021 Covid19 edition. https://...,ind,DavidBxxxx DvidThomsonB,davidbxxxx dvidthomsonb,ind
4,4,NaN,Sun Nov 21 05:03:34 +0000 2021,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1.661808e+09,I want action on climate change. Follows IA no...,whatismore,1462285537075556357,SandraC81413369,NaN,False,@sublimespirit @HelenErrington1 What choice do...,ind,whatismore I want action on climate change. Fo...,whatismore action climate change follows abc l...,ind


In [ ]:
#file1['id'] = file1['id'].astype(float)
#file1[file1['id'] == 'agsb_bilbao']

In [ ]:
#file1.dtypes

In [ ]:
#file1[file1[[ 'id', 'text']]['new_col2']=='org']
#new_df = file1[file1['new_col2']=='org'][['id', 'text']]
#new_df

In [ ]:
'''
date1 = '2021-11-21'
date2 = '2021-11-21'
#date2 = '2022-01-31'

mydates = pd.date_range(date1, date2, freq='d')#.tolist()
mydates
'''

"\ndate1 = '2021-11-21'\ndate2 = '2021-11-21'\n#date2 = '2022-01-31'\n\nmydates = pd.date_range(date1, date2, freq='d')#.tolist()\nmydates\n"

In [ ]:
date1 = '2021-11-21'
date2 = '2022-01-31'

mydates = pd.date_range(date1, date2, freq='d')#.tolist()
print(mydates)
for d in pd.date_range(date1, date2, freq='d'):
  d_str = d.strftime("%m%d%Y")
  print(d_str)

  path = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter2_res/Further_filtering/df_ff_" + d_str + ".csv"
  df = pd.read_csv(path, engine= 'python')
  df_reduced = df[df['new_col2']=='ind'][['user_id','created_at','id', 'text']]

  path2 = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter3_res/Only_ind_tweets"

  isExist = os.path.exists(path2)
  if not isExist:
  
    # Create a new directory because it does not exist 
    os.makedirs(path2)
    print("The new directory is created!")
  dest_path = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter3_res/Only_ind_tweets/df_ind_tweets_" + d_str + ".csv"
  df_reduced.to_csv(dest_path)

DatetimeIndex(['2021-11-21', '2021-11-22', '2021-11-23', '2021-11-24',
               '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28',
               '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02',
               '2021-12-03', '2021-12-04', '2021-12-05', '2021-12-06',
               '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10',
               '2021-12-11', '2021-12-12', '2021-12-13', '2021-12-14',
               '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18',
               '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22',
               '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26',
               '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30',
               '2021-12-31', '2022-01-01', '2022-01-02', '2022-01-03',
               '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
               '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11',
               '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15',
      

In [ ]:
'''
p = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Only_ind_tweets/df_ind_tweets_11262021.csv"
df_p = pd.read_csv(p, engine='python')
df_p.columns
'''

'\np = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Only_ind_tweets/df_ind_tweets_11262021.csv"\ndf_p = pd.read_csv(p, engine=\'python\')\ndf_p.columns\n'

In [ ]:
'''
date1 = '2021-11-23'
date2 = '2022-01-31'

mydates = pd.date_range(date1, date2, freq='d')#.tolist()
print(mydates)
for d in pd.date_range(date1, date2, freq='d'):
  d_str = d.strftime("%m%d%Y")
  print(d_str)
  path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further Filtered/df_ff_" + d_str + ".csv"

  df = pd.read_csv(path, engine= 'python')
  df_reduced = df[df['new_col2']=='ind'][['id', 'text']]

  path2 = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Only_ind_tweets"

  #os.makedirs(path, exist_ok=False)
  isExist = os.path.exists(path2)
  if not isExist:
  
    # Create a new directory because it does not exist 
    os.makedirs(path2)
    print("The new directory is created!")
  dest_path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Only_ind_tweets/df_ind_tweets_" + d_str + ".csv"
  df_reduced.to_csv(dest_path)

  '''



'\ndate1 = \'2021-11-23\'\ndate2 = \'2022-01-31\'\n\nmydates = pd.date_range(date1, date2, freq=\'d\')#.tolist()\nprint(mydates)\nfor d in pd.date_range(date1, date2, freq=\'d\'):\n  d_str = d.strftime("%m%d%Y")\n  print(d_str)\n  path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further Filtered/df_ff_" + d_str + ".csv"\n\n  df = pd.read_csv(path, engine= \'python\')\n  df_reduced = df[df[\'new_col2\']==\'ind\'][[\'id\', \'text\']]\n\n  path2 = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Only_ind_tweets"\n\n  #os.makedirs(path, exist_ok=False)\n  isExist = os.path.exists(path2)\n  if not isExist:\n  \n    # Create a new directory because it does not exist \n    os.makedirs(path2)\n    print("The new directory is created!")\n  dest_path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Only_ind_tweets/df_ind_tweets_" + d_str + ".csv"\n  df_reduced.to_csv(dest_path)\n\n  '

In [ ]:
'''
date1 = '2021-11-21'
date2 = '2021-11-30'

#date1 = datetime.datetime.strptime('12012021', '%m%d%Y')
#date2 = datetime.datetime.strptime('12022021', '%m%d%Y')

mydates = pd.date_range(date1, date2, freq='d')#.tolist()
print(mydates)
for d in pd.date_range(date1, date2, freq='d'):
  d_str = d.strftime("%m%d%Y")
  print(d_str)
  path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further Filtered/df_ff_" + d_str + ".csv"

  df = pd.read_csv(path, engine= 'python')
  df_reduced = df[df['new_col2']=='ind'][['id', 'text']]

  nltk.download('punkt')
  nltk.download('stopwords')

  STOP_WORDS = stopwords.words()

  # removing the emojies
  # https://www.kaggle.com/alankritamishra/covid-19-tweet-sentiment-analysis#Sentiment-analysis
  EMOJI_PATTERN = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)


  def cleaning(text):
      """
      Convert to lowercase.
      Rremove URL links, special characters and punctuation.
      Tokenize and remove stop words.
      """
      text = text.lower()
      text = re.sub('https?://\S+|www\.\S+', '', text)
      text = re.sub('@[A-Za-z0-9]+', '', text)
      text = re.sub('<.*?>+', '', text)   # replacing HTML text with blank or whitespace
      text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
      text = re.sub('\n', '', text)
      text = re.sub('[’“”…]', '', text)
      
      text = text.encode("ascii", "ignore")     # consider only english alphanumeric characters
      text = text.decode()
      
      text = EMOJI_PATTERN.sub(r'', text)
    
      
      
      # removing the stop-words
      text_tokens = word_tokenize(text)
      tokens_without_sw = [
        word for word in text_tokens if not word in STOP_WORDS]
      filtered_sentence = (" ").join(tokens_without_sw)
      text = filtered_sentence
      
      return text

  def func2(val1, val2):
    for word in uniq_list:
        if (word in val1):     # may use val2 ie user_verified status for additional filtering
            return "org"
        else:
            return "ind"

  #if __name__ == "__main__":
  #    max_rows = 1000  # 'None' to read whole file
      
  #df = pd.read_csv("Labeled_tweets.csv")
  #df.head()
  df_reduced['name_plus_desp'] = df_reduced['user_name'] + " " + df_reduced['user_description']
  df_reduced['name_plus_desp'] = df_reduced['name_plus_desp'].astype(str)
  df_reduced.head() 
  
  #dt = df_reduced['name_plus_desp'].apply(cleaning)
  df_reduced['name_plus_desp_clean'] = df_reduced['name_plus_desp'].apply(cleaning)
  
  #print(df_reduced['name_plus_desp_clean'])
  print("\n")
  
  # word_count = Counter(" ".join(dt).split()).most_common(10)
  # word_frequency = pd.DataFrame(word_count, columns = ['Word', 'Frequency'])
  # print(word_frequency)
  
  df_reduced['new_col2'] = df_reduced.apply(lambda row: func2(row['name_plus_desp_clean'], row['user_verified']), axis =1)
  print(df_reduced['new_col2'].value_counts())
  
  path2 = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further_filtering"

  #os.makedirs(path, exist_ok=False)
  isExist = os.path.exists(path2)
  if not isExist:
  
    # Create a new directory because it does not exist 
    os.makedirs(path2)
    print("The new directory is created!")
  dest_path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further_filtering/df_ff_" + d_str + ".csv"
  df_reduced.to_csv(dest_path)

'''

'\ndate1 = \'2021-11-21\'\ndate2 = \'2021-11-30\'\n\n#date1 = datetime.datetime.strptime(\'12012021\', \'%m%d%Y\')\n#date2 = datetime.datetime.strptime(\'12022021\', \'%m%d%Y\')\n\nmydates = pd.date_range(date1, date2, freq=\'d\')#.tolist()\nprint(mydates)\nfor d in pd.date_range(date1, date2, freq=\'d\'):\n  d_str = d.strftime("%m%d%Y")\n  print(d_str)\n  path = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further Filtered/df_ff_" + d_str + ".csv"\n\n  df = pd.read_csv(path, engine= \'python\')\n  df_reduced = df[df[\'new_col2\']==\'ind\'][[\'id\', \'text\']]\n\n  nltk.download(\'punkt\')\n  nltk.download(\'stopwords\')\n\n  STOP_WORDS = stopwords.words()\n\n  # removing the emojies\n  # https://www.kaggle.com/alankritamishra/covid-19-tweet-sentiment-analysis#Sentiment-analysis\n  EMOJI_PATTERN = re.compile("["\n                            u"😀-🙏"  # emoticons\n                            u"🌀-🗿"  # symbols & pictographs\n                            u"🚀-\U0001f6ff"  

In [ ]:

'''

def func2(val1, val2):
    for word in uniq_list:
        if (word in val1):     # may use val2 ie user_verified status for additional filtering
            return "org"
        else:
            return "ind"

df_reduced['new_col2'] = df_reduced.apply(lambda row: func2(row['name_plus_desp_clean'], row['user_verified']), axis =1)
'''

'\n\ndef func2(val1, val2):\n    for word in uniq_list:\n        if (word in val1):     # may use val2 ie user_verified status for additional filtering\n            return "org"\n        else:\n            return "ind"\n\ndf_reduced[\'new_col2\'] = df_reduced.apply(lambda row: func2(row[\'name_plus_desp_clean\'], row[\'user_verified\']), axis =1)\n'